In [13]:
import openai

import base64
import requests


# Set up your OpenAI API key
# set api key with openai
api_key = ...
openai.api_key = api_key


In [14]:
# Function to encode the image
def encode_image(image_path):
    with open(image_path, "rb") as image_file:
        return base64.b64encode(image_file.read()).decode('utf-8')

# Path to your image
image_path = "tom-holland-attends-the-los-angeles-premiere-of-sony-news-photo-1683915930.jpg"
prompt = """
What are the facial features of the celeberty in this image. Can you provide the following details in this structured format:
Face Shape: [Oval, Round, Square, Heart, Oblong, Diamond, Triangle]
Forehead: [Low, Medium, High]
Eyebrows: [Thin, Average, Thick, Arched, Straight]
Eyes:
  - Color: [Brown, Blue, Green, Hazel, Gray, Heterochromia]
  - Size: [Small, Medium, Large]
  - Shape: [Round, Almond, Hooded, Downturned, Upturned]
  - Distance: [Close-set, Average, Wide-set]
Nose:
  - Shape: [Straight, Concave, Convex, Button, Hooked, Bulbous]
  - Size: [Small, Medium, Large]
  - Nostril Shape: [Narrow, Average, Wide, Flared]
Cheeks: [Flat, Curved, High, Prominent]
Lips:
  - Size: [Thin, Average, Full]
  - Shape: [Narrow, Average, Wide]
  - Cupid's Bow: [Absent, Subtle, Prominent]
Chin: [Pointed, Round, Square, Dimpled, Double, Cleft]
Jawline: [Rounded, Angular, Square]
Ears: [Small, Medium, Large, Attached Earlobes, Detached Earlobes]
Hair:
  - Color: [Black, Brown, Blonde, Red, Gray, White, Dyed]
  - Texture: [Straight, Wavy, Curly, Coily]
  - Hairline: [Low, Average, High, Receding]
Estimated Age: [age range ex. late 30s]
Build: [Slim, Average, Athletic, Heavy]
Distinctive Features: [List any notable features such as freckles, dimples, scars, moles, birthmarks, tattoos, piercings, or other unique characteristics]
"""

# Getting the base64 string
base64_image = encode_image(image_path)

headers = {
    "Content-Type": "application/json",
    "Authorization": f"Bearer {api_key}"
}

payload = {
    "model": "gpt-4-vision-preview",
    "messages": [
      {
        "role": "user",
        "content": [
          {
            "type": "text",
            "text": prompt
          },
          {
            "type": "image_url",
            "image_url": {
              "url": f"data:image/jpeg;base64,{base64_image}"
            }
          }
        ]
      }
    ],
    "max_tokens": 300
}

response = requests.post("https://api.openai.com/v1/chat/completions", headers=headers, json=payload)

print(response.json())

{'id': 'chatcmpl-9I3S14dRGqn7yE93G9E74SeKGcTLI', 'object': 'chat.completion', 'created': 1714090197, 'model': 'gpt-4-1106-vision-preview', 'choices': [{'index': 0, 'message': {'role': 'assistant', 'content': "Face Shape: Oval\nForehead: Medium\nEyebrows: Average, Arched\nEyes:\n  - Color: Brown\n  - Size: Medium\n  - Shape: Almond\n  - Distance: Average\nNose:\n  - Shape: Straight\n  - Size: Medium\n  - Nostril Shape: Average\nCheeks: Curved\nLips:\n  - Size: Full\n  - Shape: Average\n  - Cupid's Bow: Prominent\nChin: Round\nJawline: Rounded\nEars: Medium, Detached Earlobes\nHair:\n  - Color: Brown\n  - Texture: Wavy\n  - Hairline: Average\nEstimated Age: Possibly in his early to mid-20s\nBuild: Slim\nDistinctive Features: None that can be clearly identified from the image provided."}, 'logprobs': None, 'finish_reason': 'stop'}], 'usage': {'prompt_tokens': 1191, 'completion_tokens': 178, 'total_tokens': 1369}, 'system_fingerprint': None}


In [15]:
facial_features = response.json()["choices"][0]['message']['content']

In [17]:
facial_features

"Face Shape: Oval\nForehead: Medium\nEyebrows: Average, Arched\nEyes:\n  - Color: Brown\n  - Size: Medium\n  - Shape: Almond\n  - Distance: Average\nNose:\n  - Shape: Straight\n  - Size: Medium\n  - Nostril Shape: Average\nCheeks: Curved\nLips:\n  - Size: Full\n  - Shape: Average\n  - Cupid's Bow: Prominent\nChin: Round\nJawline: Rounded\nEars: Medium, Detached Earlobes\nHair:\n  - Color: Brown\n  - Texture: Wavy\n  - Hairline: Average\nEstimated Age: Possibly in his early to mid-20s\nBuild: Slim\nDistinctive Features: None that can be clearly identified from the image provided."

In [18]:

# User input for facial features and image characteristics
image_characteristics = "taken with EOS R 300mm f2.8, real photo, front, zoom shot, 4k resolution, ar 9:16, v 6"
setting = "Looking to left, has glasses on"
example = '"American woman on a movie set, forty two years old, attractive, dark brown hair, taken with EOS R 300mm f2.8, real photo, front, distance shot, handsome, 4k resolution, ar 9:16, v 6, amber eyes, hair parted down middle, thin nose, realistic, casual photo"'
# Generate the prompt using the user input
prompt = f"""
Given the following facial features and image characteristics, please generate a detailed prompt which will be inputted into 
Dalle for creating an image of this person in the given setting:

Facial Features:
{facial_features}

Image Characteristics:
{image_characteristics}

Setting:
{setting}

format answer like this example: Limit answer to 70 tokens:
{example}
"""


In [19]:

# Use the OpenAI API to generate the image prompt
response = openai.ChatCompletion.create(
    model="gpt-4-turbo",
    messages=[
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": prompt}
    ],
    max_tokens=100,
    n=1,
    stop=None,
    temperature=0.7,
)


In [20]:

# Extract the generated image prompt from the API response
image_prompt = response.choices[0].message['content'].strip()

print("Generated Image Prompt:")
print(image_prompt)

Generated Image Prompt:
"Man in his early 20s with oval face, medium brown wavy hair, brown almond eyes, full lips with a prominent cupid's bow, wearing glasses, looking left, taken with EOS R 300mm f2.8, real photo, front, zoom shot, 4k resolution, ar 9:16, v 6."
